## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-09-46-48 +0000,nyt,"After Decades of Hostility, Israeli-Syrian Rel...",https://www.nytimes.com/2025/07/09/world/middl...
1,2025-07-09-09-41-48 +0000,nyt,Philadelphia Public Workers Reach Deal to End ...,https://www.nytimes.com/2025/07/09/world/ameri...
2,2025-07-09-09-36-00 +0000,wsj,Malaysia’s Central Bank Cuts Rates for First T...,https://www.wsj.com/articles/malaysias-central...
3,2025-07-09-09-32-29 +0000,bbc,At least three dead in New Mexico flash flooding,https://www.bbc.com/news/articles/cx2jk51xv65o
4,2025-07-09-09-32-12 +0000,bbc,Musk's AI firm deletes posts after chatbot pra...,https://www.bbc.com/news/articles/c4g8r34nxeno


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
82,trump,16
184,tariffs,5
26,mexico,4
46,president,4
25,new,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
51,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,45
27,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,42
52,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...,35
40,2025-07-08-23-44-53 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo,30
44,2025-07-08-23-01-40 +0000,bbc,Faisal Islam: Trump delays tariffs as the rest...,https://www.bbc.com/news/articles/c628nk51364o,29


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
51,45,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
32,27,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
27,21,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
3,17,2025-07-09-09-32-29 +0000,bbc,At least three dead in New Mexico flash flooding,https://www.bbc.com/news/articles/cx2jk51xv65o
52,15,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...
23,15,2025-07-09-04-44-58 +0000,nyt,Wildfires Burn Across Europe After Blistering ...,https://www.nytimes.com/2025/07/08/world/europ...
40,14,2025-07-08-23-44-53 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo
61,14,2025-07-08-17-12-03 +0000,nyt,The French Seaside Factory Trying to Break Chi...,https://www.nytimes.com/2025/07/08/world/europ...
66,13,2025-07-08-16-25-45 +0000,bbc,"Exhausted, angry, heartbroken: Postmasters rea...",https://www.bbc.com/news/articles/c1wp9xjey9do
17,13,2025-07-09-06-51-00 +0000,wsj,New Zealand’s Central Bank Keeps Rates on Hold...,https://www.wsj.com/articles/new-zealands-cent...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
